# **Lab 10: Database Design and Build - Case Questions**

## Compute ID: bab3fq

In [1]:
import snowflake.connector as sf
import dotenv
import os
dotenv.load_dotenv()
snowflake_password = os.getenv("snowflake_password")

In [2]:
lab_conn = sf.connect(
    user='bab3fq@virginia.edu',
    password=snowflake_password,
    account='rja95216',
    warehouse='COMPUTE_WH',
    database='DS5111_SU24',
    schema='BAB3FQ'
)

In [3]:
lab_cursor = lab_conn.cursor()

## **Case Question 1**
### Which courses are currently included (active) in the program? Include the course mnemonic and course name for each. 

In [4]:
case_1_query = '''
SELECT
mnemonic,
course_name
FROM COURSES_ERD
WHERE active = 'TRUE';
'''

lab_cursor.execute(case_1_query)
case_1_df = lab_cursor.fetch_pandas_all()
case_1_df

,MNEMONIC,COURSE_NAME
0,ds_biz_anaytics,Business Analytics for Data Scientists
1,ds_tech_bootcamp,Technical Bootcamp
2,ds5001,Exploratory Text Analytics
3,ds5012,Foundations of Computer Science
4,ds5100,Programming for Data Science
5,ds5110,Big Data Systems
6,ds6001,Practice and Application of Data Science
7,ds6002,Big Data Ethics
8,ds6011,Data Science Capstone Project Work I
9,ds6013,Data Science Capstone Project Work II


## Case Question 2
### **Which courses were included in the program, but are no longer active? Include the course mnemonic and course name for each.**

In [5]:
case_2_query = '''
SELECT
mnemonic,
course_name
FROM courses_erd
WHERE active = 'FALSE';
'''
lab_cursor.execute(case_2_query)
case_2_df = lab_cursor.fetch_pandas_all()
case_2_df

,MNEMONIC,COURSE_NAME
0,ds6003,Practice and Application of Data Science Part 2
1,ds6012,Big Data Ethics Part 2


## Case Question 3
### **Which instructors are not current employees?**

In [6]:
case_3_query = '''
SELECT
instructor_name
FROM instructors_erd
WHERE active = 'FALSE';
'''
lab_cursor.execute(case_3_query)
case_3_df = lab_cursor.fetch_pandas_all()
case_3_df

,INSTRUCTOR_NAME
0,Jeremy Bolton
1,Luis Felipe Rosado Murillo


## Case Question 4
### **For each course (active and inactive), how many learning outcomes are there?**

In [7]:
case_4_query = '''
SELECT
c.course_name,
COUNT(DISTINCT lo.learning_outcome) AS Learning_Outcome_Count
FROM LEARNING_OUTCOMES_ERD lo
LEFT JOIN COURSES_ERD  c
ON lo.course_id = c.course_id
GROUP BY c.course_name
ORDER BY Learning_Outcome_Count DESC;
'''
lab_cursor.execute(case_4_query)
case_4_df = lab_cursor.fetch_pandas_all()
case_4_df

,COURSE_NAME,LEARNING_OUTCOME_COUNT
0,Big Data Systems,37
1,Bayesian Machine Learning,37
2,Programming for Data Science,34
3,Deep Learning,24
4,Statistical Learning,19
5,Technical Bootcamp,15
6,Linear Models for Data Science,13
7,Practice and Application of Data Science,10
8,Business Analytics for Data Scientists,9
9,Foundations of Computer Science,9


## Case Question 5
### **Are there any courses with no learning outcomes? If so, provide their mnemonics and names.**

In [8]:
case_5_query = '''
SELECT
c.mnemonic,
c.course_name
FROM COURSES_ERD c
LEFT JOIN LEARNING_OUTCOMES_ERD lo
ON c.course_id = lo.course_id
WHERE lo.course_id IS NULL;
'''
lab_cursor.execute(case_5_query)
case_5_df = lab_cursor.fetch_pandas_all()
case_5_df

,MNEMONIC,COURSE_NAME
0,sarc5400,Data Visualization


## Case Question 6
### **Which courses include SQL as a learning outcome? Include the learning outcome descriptions, course mnemonics, and course names in your solution.**

In [9]:
case_6_query = '''
SELECT
c.mnemonic,
c.course_name,
lo.learning_outcome
FROM LEARNING_OUTCOMES_ERD lo
LEFT JOIN COURSES_ERD  c
ON lo.course_id = c.course_id
WHERE
lo.learning_outcome LIKE '%SQL%';
'''
lab_cursor.execute(case_6_query)
case_6_df = lab_cursor.fetch_pandas_all()
case_6_df

,MNEMONIC,COURSE_NAME,LEARNING_OUTCOME
0,ds5110,Big Data Systems,Apply Spark SQL to data analysis tasks
1,ds6001,Practice and Application of Data Science,"Understand the purpose, typology, and language..."
2,ds6001,Practice and Application of Data Science,Understand how to implement databases Python: ...
3,ds6001,Practice and Application of Data Science,Understand how to query databases with SQL


## Case Question 7
### **Who taught course ds5100 in Summer 2021?**

In [10]:
case_7_query = '''
SELECT
i.instructor_name
FROM COURSE_ASSIGNMENT_ERD ca
LEFT JOIN instructors_erd i
ON ca.instructor_id = i.instructor_id
LEFT JOIN courses_erd c
ON ca.course_id = c.course_id
WHERE
c.mnemonic = 'ds5100'
AND ca.term = 'summer2021';
'''
lab_cursor.execute(case_7_query)
case_7_df = lab_cursor.fetch_pandas_all()
case_7_df

,INSTRUCTOR_NAME
0,Nada Basit


## Case Question 8
### **Which instructors taught in Fall 2021? Order their names alphabetically, making sure the names are unique.**

In [11]:
case_8_query = '''
SELECT DISTINCT
i.instructor_name
FROM COURSE_ASSIGNMENT_ERD ca
LEFT JOIN INSTRUCTORS_ERD i
ON ca.instructor_id = i.instructor_id
WHERE ca.term = 'fall2021'
ORDER BY i.instructor_name;
'''
lab_cursor.execute(case_8_query)
case_8_df = lab_cursor.fetch_pandas_all()
case_8_df

,INSTRUCTOR_NAME
0,Abbas Kazemipour
1,Adam Tashman
2,Bill Basener
3,Cait Dreisbach
4,Eric Tassone
5,Jeffrey Woo
6,Jeremy Bolton
7,Judy Fox
8,Lei Xie
9,Marc Ruggiano


## Case Question 9
### **How many courses did each instructor teach in each term? Order your results by term and then instructor.**

In [12]:
case_9_query = '''
SELECT
i.instructor_name,
ca.term,
COUNT(ca.assignment_id) AS course_count
FROM course_assignment_erd ca
LEFT JOIN instructors_erd i
on ca.instructor_id = i.instructor_id
GROUP BY ca.term, i.instructor_name
ORDER BY ca.term, instructor_name;
'''
lab_cursor.execute(case_9_query)
case_9_df = lab_cursor.fetch_pandas_all()
case_9_df

,INSTRUCTOR_NAME,TERM,COURSE_COUNT
0,Abbas Kazemipour,fall2021,1
1,Adam Tashman,fall2021,1
2,Bill Basener,fall2021,2
3,Cait Dreisbach,fall2021,1
4,Eric Tassone,fall2021,2
5,Jeffrey Woo,fall2021,1
6,Jeremy Bolton,fall2021,2
7,Judy Fox,fall2021,1
8,Lei Xie,fall2021,1
9,Marc Ruggiano,fall2021,1


## Case Question 10a
## **Which courses had more than one instructor for the same term? Provide the mnemonic and term for each. Note this occurs in courses with multiple sections.**

In [13]:
case_10a_query = '''
SELECT
c.mnemonic,
ca.term
FROM course_assignment_erd ca
LEFT JOIN courses_erd c
ON ca.course_id = c.course_id
GROUP BY c.mnemonic, ca.term
HAVING COUNT(DISTINCT ca.instructor_id) > 1
ORDER BY ca.term, c.mnemonic;
'''
lab_cursor.execute(case_10a_query)
case_10a_df = lab_cursor.fetch_pandas_all()
case_10a_df

,MNEMONIC,TERM
0,ds6011,fall2021
1,ds6013,fall2021
2,ds6011,summer2021


## Case Question 10b
### **For courses with multiple sections, provide the term, course mnemonic, and instructor name for each. Hint: You can use your result from 10a in a subquery or WITH clause.**

In [14]:
case_10b_query = '''
WITH courses_multiple_instructors AS (
    SELECT
    c.mnemonic,
    ca.term,
    ca.course_id
    FROM course_assignment_erd ca
    LEFT JOIN courses_erd c
    ON ca.course_id = c.course_id
    GROUP BY c.mnemonic, ca.term, ca.course_id
    HAVING COUNT(DISTINCT ca.instructor_id) > 1
)
SELECT
cmi.term,
cmi.mnemonic,
i.instructor_name
FROM courses_multiple_instructors cmi
LEFT JOIN COURSE_ASSIGNMENT_ERD ca
on cmi.course_id = ca.course_id
AND cmi.term = ca.term
LEFT JOIN INSTRUCTORS_ERD i
ON ca.instructor_id = i.instructor_id
ORDER BY cmi.term, cmi.mnemonic, i.instructor_name;
'''
lab_cursor.execute(case_10b_query)
case_10b_df = lab_cursor.fetch_pandas_all()
case_10b_df

,TERM,MNEMONIC,INSTRUCTOR_NAME
0,fall2021,ds6011,Abbas Kazemipour
1,fall2021,ds6011,Cait Dreisbach
2,fall2021,ds6011,Eric Tassone
3,fall2021,ds6011,Jeremy Bolton
4,fall2021,ds6011,Lei Xie
5,fall2021,ds6011,Peter Gedeck
6,fall2021,ds6013,Raf Alvarado
7,fall2021,ds6013,Teague Henry
8,summer2021,ds6011,Raf Alvarado
9,summer2021,ds6011,Teague Henry


In [15]:
lab_conn.close()